In [67]:
# https://medium.com/@a173823/web-scraping-on-shopee-com-6180979eff78
# Regulatron01!
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from collections import Counter
from tqdm import tqdm
import time

# import threading

# import pandas as pd
# import numpy as np
# from numpy import nan
# import re
# import concurrent.futures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math

In [79]:
# create object for chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') 
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()),
                              options = chrome_options)  
driver.implicitly_wait(5)
actions = ActionChains(driver)

# Maximiza a janela
driver.maximize_window()
# navegar até a shopee
driver.get('https://shopee.com.br/')
time.sleep(3)
elements = driver.find_elements(By.TAG_NAME, 'button')

for e in elements:
    if e.text == 'Aceitar todos os cookies':
        actions.move_to_element(e)
        actions.perform()
        e.click()
        break
        
element = driver.find_element(By.ID, 'main')
actions.move_to_element(element).click().perform()
time.sleep(3)
# clicar para entrar
e = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Entre')[0]
actions.move_to_element(e).click().perform()



# fazer login
telefone = '71999243010'
senha    = 'Regulatron01!'

campo_telefone = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[0]
campo_senha = driver.find_elements(By.CLASS_NAME, 'pDzPRp')[1]

actions.move_to_element(campo_telefone).click().perform()
for i in telefone:
    campo_telefone.send_keys(i)
    time.sleep(0.5)

actions.move_to_element(campo_senha).click().perform()
for i in senha:
    campo_senha.send_keys(i)
    time.sleep(0.5)

# clicar para entrar
elements = driver.find_elements(By.TAG_NAME, 'button')
for e in elements:
    if e.text == 'ENTRE':
        actions.move_to_element(e).click().perform()
        break
        
time.sleep(3)


# Pesquisar
links = set()
query = 'tvbox'
search_box = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div[2]/div/div[2]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
actions.move_to_element(search_box).click().perform()
time.sleep(0.5)

for i in query:
    search_box.send_keys(i)
    time.sleep(0.5)

button = driver.find_element(By.CSS_SELECTOR, '#main > div > div:nth-child(3) > div.Vdye40 > div > div:nth-child(2) > div > div:nth-child(2) > div > header > div.container-wrapper.header-with-search-wrapper > div > div.header-with-search__search-section > div.shopee-searchbar > button > svg')
actions.move_to_element(button).click().perform()
time.sleep(3)


try:
    ultima_pagina = int(driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[2]/div/span[2]').text)
except:
    ultima_pagina = 1

for i in range(ultima_pagina):

    # rolagem para esperar carregar os produtos
    for i in range(50):
        try:
            driver.execute_script("window.scrollBy(0, 100)")
            time.sleep(0.5)
        except:
            break


    time.sleep(3)
    div_links = driver.find_element(By.CSS_SELECTOR, '#main > div > div.dYFPlI > div > div > div.sdzgsX > div.shopee-search-item-result > div.row.shopee-search-item-result__items')
    a_elements = div_links.find_elements(By.TAG_NAME, 'a')

    for a in a_elements:
        links.add(a.get_attribute('href'))

    button = driver.find_elements(By.CLASS_NAME, 'shopee-icon-button--right')[0]
    actions.move_to_element(button).click().perform()
    time.sleep(3)

In [80]:
len(links)

1015

In [53]:
for i in links:
    driver.get(i)
    time.sleep(3)
    

In [81]:
driver.get(list(links)[0])
sleep(3)

In [83]:
    dict_produtos = { # vai guardar todas as informações dos produtos
        'produto_pesquisado' : [],
        'titulo_produto' : [],
        # 'marca' : marca,
        'id_vendedor': [],
        'preco': [],
        'quantidade': [],
        'descricao': [] ,
        'homologado': [],
        'plataforma' : [],
        'url' : []
        # 'quantidade_vendida':[]
        # modelo' : modelo
    }
    
    

In [98]:
try:
    titulo_produto = driver.find_element(By.XPATH,
                                         '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]'
                                        ).text
except:
    titulo_produto = ''

try:
    id_vendedor    = driver.find_element(By.XPATH,
                                         '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[1]/div[1]/div/div[1]'
                                        ).text
except:
    id_vendedor    = ''

try:      
    qtde_vendida = driver.find_element(By.XPATH,
                                       '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[2]/div[2]/div[1]'
                                      ).text
except:
    qtde_vendida = '0'

try:
    localizacao = driver.find_element(By.XPATH,
                                        '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[4]/div/div[2]/div/div[2]/div[2]/div[1]/div[2]/div[1]'
                                     ).text
except:
    localizacao = ''
    

try:
    preco = driver.find_element(By.XPATH,
                               '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[1]'
                               ).text

    if preco.find('-'):
        preco = preco.split('-')
        preco_minimo = preco[0][2:]
        preco_maximo = preco[-1][3:]
    else:
        preco_minimo = preco[2:]
        preco_maximo = ''
except:
    preco_minimo = ''
    preco_maximo = ''


try:
    preco_promo = driver.find_element(By.XPATH,
                               '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[3]/div/div/div/div/div[2]/div[1]'
                               ).text

    if preco_promo.find('-'):
        preco_promo = preco_promo.split('-')
        preco_promo_minimo = preco_promo[0][2:]
        preco_promo_maximo = preco_promo[-1][3:]
    else:
        preco_promo_minimo = preco_promo[2:]
        preco_promo_maximo = ''

except:
        preco_promo_minimo = ''
        preco_promo_maximo = ''

        
try:
    detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                          ).get_attribute('innerHTML')
except:
    detalhes_produto = ''
    
print(detalhes_produto)
#         driver.find_element(By.XPATH,        
# <button class="product-variation" aria-label="2G 16G" aria-disabled="false">2G 16G</button>

# try:
#     descricao = driver.find_element(By.ID, 'productDescription').text
# except:
#     descricao = ''
    
# quantidade = 1

# # if testar_palavras(f'{titulo_produto} {descricao}', yes_words, no_words):
# #     homologado = 'CANDIDATO'
# # else:
# #     homologado = ''
# homologado = ''
    
# dict_produtos['produto_pesquisado'].append(query)
# dict_produtos['titulo_produto'].append(titulo_produto)
# # dict_produtos['marca'].append(marca)
# dict_produtos['id_vendedor'].append(id_vendedor)
# dict_produtos['preco'].append(preco)
# dict_produtos['quantidade'].append(quantidade)
# dict_produtos['descricao'].append(descricao)
# dict_produtos['homologado'].append(homologado)
# dict_produtos['plataforma'].append('Carrefour')
# dict_produtos['url'].append(list(links)[0])
# # dict_produtos['modelo'].append(modelo)

# dict_produtos
# #salvar_dict_para_csv(dict_produtos, 'dados/resultado_mercado_livre.csv')

<div class="dR8kXc"><label class="zquA4o">Categoria</label><div class="flex items-center RnKf-X"><a class="akCPfg KvmvO1" href="/">Shopee</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Jogos-e-Consoles-cat.11059980">Jogos e Consoles</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Acessórios-de-Consoles-cat.11059980.11060087">Acessórios de Consoles</a></div></div><div class="dR8kXc"><label class="zquA4o eqeCX7">Compatilbilidade da Plataforma</label><div>Para Celul

In [116]:
try:
    detalhes_produto = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[1]/div/div/div/div[3]/div[2]/div[1]/div[1]/div[1]/div[2]'
                                          ).get_attribute('innerHTML')
except:
    detalhes_produto = ''
    
print(detalhes_produto)

<div class="dR8kXc"><label class="zquA4o">Categoria</label><div class="flex items-center RnKf-X"><a class="akCPfg KvmvO1" href="/">Shopee</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Jogos-e-Consoles-cat.11059980">Jogos e Consoles</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Acessórios-de-Consoles-cat.11059980.11060087">Acessórios de Consoles</a></div></div><div class="dR8kXc"><label class="zquA4o eqeCX7">Compatilbilidade da Plataforma</label><div>Para Celul

In [120]:
import re

padrao = re.findall(r'<label[^>]*>(.*?)</label>', detalhes_produto)
print(padrao)

['Categoria', 'Compatilbilidade da Plataforma', 'Quantidade', 'Estoque', 'Enviado de']


In [138]:
# detalhes_produto

# criar um objeto Beautiful Soup a partir do HTML
soup = BeautifulSoup(detalhes_produto, 'html.parser')

# remover todas as tags <label> e svg
for label in soup.find_all(['label','svg']):
    label.extract()
    
for div in soup.find_all('div', {'class': 'dR8kXc'}):
    div.unwrap()


# seleciona todas as tags <div> e extrai o texto
divs = [div.text.replace('\n',' ').strip() for div in soup.find_all('div')]

# imprime a lista resultante
print(divs)
    
    
    
    
    
# # imprimir o HTML sem as tags <label>
# print(soup.prettify())

['ShopeeJogos e ConsolesAcessórios de Consoles', 'Para Celular', 'Nil', '2400000', 'China Continental']


In [124]:
padrao = re.findall(r'<div[^>]*>(.*?)</div>', detalhes_produto)
print(padrao)

['<label class="zquA4o">Categoria</label><div class="flex items-center RnKf-X"><a class="akCPfg KvmvO1" href="/">Shopee</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Jogos-e-Consoles-cat.11059980">Jogos e Consoles</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Acessórios-de-Consoles-cat.11059980.11060087">Acessórios de Consoles</a>', '<label class="zquA4o eqeCX7">Compatilbilidade da Plataforma</label><div>Para Celular', '<label class="zquA4o eqeCX7">Quantidade<

In [106]:
detalhes_produto

'<div class="dR8kXc"><label class="zquA4o">Categoria</label><div class="flex items-center RnKf-X"><a class="akCPfg KvmvO1" href="/">Shopee</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Jogos-e-Consoles-cat.11059980">Jogos e Consoles</a><svg enable-background="new 0 0 11 11" viewBox="0 0 11 11" x="0" y="0" class="shopee-svg-icon xprSzi icon-arrow-right"><path d="m2.5 11c .1 0 .2 0 .3-.1l6-5c .1-.1.2-.3.2-.4s-.1-.3-.2-.4l-6-5c-.2-.2-.5-.1-.7.1s-.1.5.1.7l5.5 4.6-5.5 4.6c-.2.2-.2.5-.1.7.1.1.3.2.4.2z"></path></svg><a class="akCPfg KvmvO1" href="/Acessórios-de-Consoles-cat.11059980.11060087">Acessórios de Consoles</a></div></div><div class="dR8kXc"><label class="zquA4o eqeCX7">Compatilbilidade da Plataforma</label><div>Para Celu

In [ ]:
links = set()
driver.get('https://www.amazon.com.br')
sleep(3)

search(driver = driver, query = query, element_id = element_id)
sleep(3)

## descobre a úlitim página 
# <span class="s-pagination-item s-pagination-disabled" aria-disabled="true">7</span>
try:
    ultima_pagina = driver.find_elements(By.CLASS_NAME, 's-pagination-disabled')[1].get_attribute('innerHTML')
    ultima_pagina = int(ultima_pagina)
except: 
    ultima_pagina = 1
    

for pagina in range(ultima_pagina):
    try:
        print('pesquisando página ', pagina + 1, 'de', ultima_pagina)
        #descobrir os links dos produtos
        
        a_elements = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
        # a_elements = a_elements.find_elements(By.TAG_NAME, 'a')
        for element in a_elements:
            link_produto = element.get_attribute('href') 
#             16,https://www.amazon.com.br/gp/goldbox/
#             ,https://www.amazon.com.br/s?k=
            if '/gp/' in link_produto:
                pass
            elif '/s?' in link_produto:
                pass 
            elif '/s/' in link_produto:
                pass
            elif '/b?' in link_produto:
                pass
            else:
                links.add(link_produto)

        if pagina < ultima_pagina:
            driver.find_elements(By.CLASS_NAME, 's-pagination-next')[0].click()
            sleep(3)
            
    except Exception as e:
        print(e)


In [ ]:
# for link in links()
#     driver.get(link)
#     sleep(3)
    


In [ ]:
driver.get(list(links)[0])
sleep(3)

In [ ]:
try:
    titulo_produto = driver.find_element(By.ID, 'productTitle').text
except:
    titulo_produto = ''

try:
    id_vendedor    = driver.find_element(By.XPATH, '//*[@id="tabular-buybox"]/div[1]/div[6]/div').get_attribute('innerText')

except:
    id_vendedor    = ''

try:      

    string = driver.find_element(By.CLASS_NAME, 'a-price-whole').get_attribute('innerText')
    parte_numerica = re.findall(r'\d+', string)
    parte_inteira = int(parte_numerica[0])
    
    string = driver.find_element(By.CLASS_NAME, 'a-price-fraction').get_attribute('innerText')
    parte_numerica = re.findall(r'\d+', string)
    parte_decimal = int(parte_numerica[0])

    preco = float(f'{parte_inteira}.{parte_decimal}')
    preco = '{:0.2f}'.format(preco).replace('.', ',') 

except:
    preco = ''

try:
    descricao = driver.find_element(By.ID, 'productDescription').text
except:
    descricao = ''
    
quantidade = 1

# if testar_palavras(f'{titulo_produto} {descricao}', yes_words, no_words):
#     homologado = 'CANDIDATO'
# else:
#     homologado = ''
homologado = ''
    
dict_produtos['produto_pesquisado'].append(query)
dict_produtos['titulo_produto'].append(titulo_produto)
# dict_produtos['marca'].append(marca)
dict_produtos['id_vendedor'].append(id_vendedor)
dict_produtos['preco'].append(preco)
dict_produtos['quantidade'].append(quantidade)
dict_produtos['descricao'].append(descricao)
dict_produtos['homologado'].append(homologado)
dict_produtos['plataforma'].append('Carrefour')
dict_produtos['url'].append(list(links)[0])
# dict_produtos['modelo'].append(modelo)

dict_produtos
#salvar_dict_para_csv(dict_produtos, 'dados/resultado_mercado_livre.csv')

In [ ]:


descricao

In [ ]:
descricao = find_element(By.ID, 'productDescription')

In [ ]:
            dict_produtos['produto_pesquisado'].append(chave)
            dict_produtos['titulo_produto'].append(titulo_produto) 
            # dict_produtos['marca'].append(marca)
            dict_produtos['id_vendedor'].append(id_vendedor)
            dict_produtos['preco'].append(preco)
            dict_produtos['quantidade'].append(quantidade)
            dict_produtos['descricao'].append(descricao)
            dict_produtos['homologado'].append(homologado)
            dict_produtos['plataforma'].append('Mercado Livre')
            dict_produtos['url'].append(item)
            # dict_produtos['modelo'].append(modelo)
            
<span id="productTitle" class="a-size-large product-title-word-break">        Controle Remoto Universal Wifi Smart Home Elgin Compatível com os Sistemas Amazon Alexa e Google Home, Vermelho       </span>